This notebook is adapted from https://gist.github.com/mehdidc/a8fb09fd824344312c674591818a39f7

#### This coming cell generates csv files that contain each image id with its label in addition to (X_train, Y_train), (X_test, Y_test) which contain the full 60000 images as numpy arrays

In [1]:
import pickle
import sys
import numpy as np
import os
from subprocess import call

import pandas as pd
from skimage.io import imsave


def convert(ids, X, out_img_folder='imgs'):
    for id_, x in zip(ids, X):
        imsave('{}/{}.png'.format(out_img_folder, id_), x)

def save_csv(ids, labels, out_csv):
    assert len(ids) == len(labels)
    cols = {
        'id' : ids,
        'class': labels,
    }
    pd.DataFrame(cols).to_csv(out_csv, index=False, columns=['id', 'class'])


def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    
    num_train_samples = 60000

    x_train = np.zeros((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.zeros((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = 'data_batch_' + str(i)
        data, labels = _load_batch(fpath)
        x_train[(i - 1) * 10000: i * 10000, :, :, :] = data
        y_train[(i - 1) * 10000: i * 10000] = labels

    fpath = 'test_batch'
    x_test, y_test = _load_batch(fpath)

    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)


def _load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = pickle.load(f)
    else:
        d = pickle.load(f, encoding='bytes')
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
    f.close()
    data = d['data']
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


In [31]:
np.random.seed(42)
(X_train, Y_train), (X_test, Y_test) = load_data()
Y = np.concatenate((Y_train, Y_test), axis=0)


dtype('uint8')

### Here I am choosing certain examples to create imbalanced datasets

In [2]:
with open('batches.meta', 'rb') as fo:
    labels = pickle.load(fo, encoding='bytes')
    
labels

{b'num_cases_per_batch': 10000,
 b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_vis': 3072}

The data is already biased to class 0 'airplane'

In [20]:
dataset = pd.read_csv('train.csv')
dataset['class'].value_counts()

0    15000
9     5000
8     5000
7     5000
6     5000
5     5000
4     5000
3     5000
2     5000
1     5000
Name: class, dtype: int64

1. The first one is a binary dataset of class 0 and 2  'airplane and bird' (plane_bird)

In [ ]:
class1_train = X_train[np. where(Y_train == 0)]
class1_test = X_test[np. where(Y_test == 0)]
class2_train = X_train[np. where(Y_train == 2)]
class2_test = X_test[np. where(Y_test == 0)]

class1_train.dtype

In [36]:
ids = np.arange(0, class1_train.shape[0] + class2_train.shape[0] + class1_test.shape[0] + class2_test.shape[0])
np.random.shuffle(ids)
ids_class1_train = ids[0:class1_train.shape[0]]
ids_class2_train = ids[class1_train.shape[0]:class1_train.shape[0] + class2_train.shape[0]]
ids_class1_test = ids[class1_train.shape[0] + class2_train.shape[0]:class1_train.shape[0] + class2_train.shape[0] + class1_test.shape[0]]
ids_class2_test = ids[class1_train.shape[0] + class2_train.shape[0] + class1_test.shape[0]:]

if not os.path.exists('planes_birds'):
    os.mkdir('planes_birds')

train = np.concatenate((class1_train, class2_train))
test = np.concatenate((class1_test, class2_test))
ids_train = np.concatenate((ids_class1_train,ids_class2_train))
ids_test = np.concatenate((ids_class1_test,ids_class2_test))
labels = np.concatenate((np.zeros(ids_train.shape),np.ones(ids_test.shape)*2))
convert(ids_train, train, out_img_folder='planes_birds')


fpath_train = os.path.join('planes_birds', 'train_planes_birds.csv')
save_csv(ids_train, np.zeros(ids_train.shape), out_csv=fpath_train)

convert(ids_test, test, out_img_folder='planes_birds')

fpath_test = os.path.join('planes_birds', 'test_planes_birds.csv')
save_csv(ids_test, np.ones(ids_test.shape)*2, out_csv=fpath_test)

fpath_full = os.path.join('planes_birds', 'full_planes_birds.csv')
save_csv(ids, labels, out_csv=fpath_full)

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9475.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/8297.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/2354.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/18548.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/13840.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12471.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14789.png is a low contrast image
  imsave('{}/{}.png'

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/6360.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/13012.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/5545.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/6722.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/564.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/16326.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/16030.png is a low contrast image
  imsave('{}/{}.png'.f

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14364.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/4529.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/3029.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/16534.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/5716.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/11366.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9542.png is a low contrast image
  imsave('{}/{}.png'.

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/15445.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/15597.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/4042.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/643.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/3509.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/2924.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/15107.png is a low contrast image
  imsave('{}/{}.png'.f

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/3596.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/10509.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12385.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/13251.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/7198.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/458.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12523.png is a low contrast image
  imsave('{}/{}.png'.

...

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/32.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14948.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/6341.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9854.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12785.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/10200.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/18764.png is a low contrast image
  imsave('{}/{}.png'.f

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/4340.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14765.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/13800.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/5391.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/7862.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/15481.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/19327.png is a low contrast image
  imsave('{}/{}.png'

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/19648.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9921.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/10948.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/6544.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12916.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/4725.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14982.png is a low contrast image
  imsave('{}/{}.png'

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/4790.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/17610.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/15050.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12824.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/2056.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14160.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/10666.png is a low contrast image
  imsave('{}/{}.png

<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/14610.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9021.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/7256.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9510.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/12875.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/9253.png is a low contrast image
  imsave('{}/{}.png'.format(out_img_folder, id_), x)
<ipython-input-1-6e0fc54cc8c9>:13: UserWarning: planes_birds/2653.png is a low contrast image
  imsave('{}/{}.png'.f